<center> <h1>Local Embeddings on CPU with LM Studio</h1>

## 1. Install langchain-openai & langchain-qdrant packages

In [1]:
!pip install -U langchain-openai langchain-qdrant

  Using cached langchain_openai-0.1.23-py3-none-any.whl (51 kB)
  Using cached langchain_qdrant-0.1.3-py3-none-any.whl (22 kB)


## 2. Instanciate an OpenAIEmbeddings object with the LM Studio base url

### 2.1 Local URL

In [2]:
from langchain_openai import OpenAIEmbeddings

model = OpenAIEmbeddings(
    openai_api_base="http://localhost:1234/v1", 
    api_key="type-anything-here", 
    check_embedding_ctx_length=False
)

In [3]:
query = "this is my first lm studio embedding input"
vector = model.embed_query(query)
print(len(vector))

768


### 2.2 Public URL (e.g. for colab users)

As colab raises a Connection Error when calling a "localhost" endpoint, you can turn it into a public url with [Ngrok](https://ngrok.com/).

In [4]:
model_bis = OpenAIEmbeddings(
    openai_api_base="https://3646-46-193-69-6.ngrok-free.app/v1", 
    api_key="type-anything-here", 
    check_embedding_ctx_length=False
)

In [5]:
vector_bis = model_bis.embed_query(query)
print(vector==vector_bis)

True


## 3. Create a [Qdrant](https://qdrant.tech/#) collection for your Embedding model

In [6]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(path="/tmp/tutorials")

client.create_collection(
    collection_name="embedding_tutorial",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

True

## 4. Embed & upsert documents to the Vector Store

In [7]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="search_document: I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
        metadata={"source": "tweet"},
        id=1,
    ),
     Document(
        page_content="search_document: The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
        metadata={"source": "news"},
        id=2,
    ),
    Document(
        page_content="search_document: Building an exciting new project with LangChain - come check it out!",
        metadata={"source": "tweet"},
        id=3,
    ),
    Document(
        page_content="search_document: Robbers broke into the city bank and stole $1 million in cash.",
        metadata={"source": "news"},
        id=4,
    ),
    Document(
        page_content="search_document: Wow! That was an amazing movie. I can't wait to see it again.",
        metadata={"source": "tweet"},
        id=5,
    ),
    Document(
        page_content="search_document: Is the new iPhone worth the price? Read this review to find out.",
        metadata={"source": "website"},
        id=6,
    ),
    Document(
        page_content="search_document: The top 10 soccer players in the world right now.",
        metadata={"source": "website"},
        id=7,
    ),
    Document(
        page_content="search_document: LangGraph is the best framework for building stateful, agentic applications!",
        metadata={"source": "tweet"},
        id=8,
    ),
    Document(
        page_content="search_document: The stock market is down 500 points today due to fears of a recession.",
        metadata={"source": "news"},
        id=9,
    ),
    Document(
        page_content="search_document: I have a bad feeling I am going to get deleted :(",
        metadata={"source": "tweet"},
        id=10,
    )
]

In [8]:
from langchain_qdrant import QdrantVectorStore, RetrievalMode

qdrant = QdrantVectorStore.from_documents(
    documents,
    embedding=model,
    location=":memory:",
    collection_name="embedding_tutorial",
    retrieval_mode=RetrievalMode.DENSE,
)

## 5. Perform similarity search with metadata filtering

In [9]:
from qdrant_client.http import models

def retrieve(query, top_k, filt):
    key, value = filt.split("==")
    results = qdrant.similarity_search_with_score(
        f"search_query: {query}", k=top_k, filter=models.Filter(
            should=[
                models.FieldCondition(
                    key=key,
                    match=models.MatchValue(
                        value=value
                    ),
                ),
            ]
        ),
    )
    for res, score in results:
        print(f"*Text: {res.page_content[17:]}\n Source: {res.metadata['source']}\n Score: {score}\n\n")
    return results

In [10]:
results = retrieve(query="what did you think about the movie ?", top_k=5, filt="metadata.source==tweet")

*Text: Wow! That was an amazing movie. I can't wait to see it again.
 Source: tweet
 Score: 0.37506646278842126


*Text: LangGraph is the best framework for building stateful, agentic applications!
 Source: tweet
 Score: 0.3251572418324922


*Text: I have a bad feeling I am going to get deleted :(
 Source: tweet
 Score: 0.2657674549552088


*Text: I had chocalate chip pancakes and scrambled eggs for breakfast this morning.
 Source: tweet
 Score: 0.2270696175199384


*Text: Building an exciting new project with LangChain - come check it out!
 Source: tweet
 Score: 0.15759966780919388


